# California House Prices Prediction v2 (AutoML + LightGBM)

**Kaggle Competition**: [California House Prices](https://www.kaggle.com/competitions/california-house-prices/overview)

이 노트북에서는 AutoML 방식(Optuna)과 LightGBM을 활용하여 캘리포니아 주택 가격을 예측합니다.

## v2 개선사항
- Optuna를 활용한 자동 하이퍼파라미터 튜닝
- LightGBM 중심의 최적화된 모델링
- 향상된 특성 엔지니어링
- Target Encoding 적용
- Stratified K-Fold 교차 검증

## 목차
1. 라이브러리 및 데이터 로드
2. 탐색적 데이터 분석 (EDA)
3. 데이터 전처리
4. 특성 엔지니어링
5. AutoML - Optuna 하이퍼파라미터 튜닝
6. 최종 모델 학습 및 예측
7. 제출 파일 생성

## 1. 라이브러리 및 데이터 로드

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# 전처리 및 모델링
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 모델
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

# AutoML - Optuna
import optuna
from optuna.integration import LightGBMPruningCallback

# 시각화 설정
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# 재현성
SEED = 42
np.random.seed(SEED)

print('Libraries loaded successfully!')

In [ ]:
# 데이터 로드
DATA_PATH = 'data/california-house-prices/'

train = pd.read_csv(DATA_PATH + 'train.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')
submission = pd.read_csv(DATA_PATH + 'sample_submission.csv')

print(f'Train shape: {train.shape}')
print(f'Test shape: {test.shape}')
print(f'Submission shape: {submission.shape}')

In [ ]:
# 데이터 미리보기
train.head()

### 컬럼 설명 (Data Dictionary)

| 컬럼명 | 설명 | 타입 |
|--------|------|------|
| **Id** | 고유 식별자 | int |
| **Address** | 주택 주소 | str |
| **Sold Price** | 판매 가격 (USD) - **타겟 변수** | float |
| **Summary** | 매물 설명 텍스트 | str |
| **Type** | 주택 유형 (SingleFamily, Condo, Townhouse, VacantLand 등) | str |
| **Year built** | 건축 연도 | float |
| **Heating** | 난방 시스템 유형 | str |
| **Cooling** | 냉방 시스템 유형 | str |
| **Parking** | 주차 시설 유형 | str |
| **Lot** | 대지 면적 (평방피트, sqft) | float |
| **Bedrooms** | 침실 수 | object |
| **Bathrooms** | 욕실 수 | float |
| **Full bathrooms** | 완전한 욕실 수 (샤워/욕조 포함) | float |
| **Total interior livable area** | 실내 거주 가능 면적 (sqft) | float |
| **Total spaces** | 총 주차 공간 수 | float |
| **Garage spaces** | 차고 주차 공간 수 | float |
| **Region** | 지역/도시명 | str |
| **Elementary School** | 인근 초등학교 이름 | str |
| **Elementary School Score** | 초등학교 평점 (1-10) | float |
| **Elementary School Distance** | 초등학교까지 거리 (마일) | float |
| **Middle School** | 인근 중학교 이름 | str |
| **Middle School Score** | 중학교 평점 (1-10) | float |
| **Middle School Distance** | 중학교까지 거리 (마일) | float |
| **High School** | 인근 고등학교 이름 | str |
| **High School Score** | 고등학교 평점 (1-10) | float |
| **High School Distance** | 고등학교까지 거리 (마일) | float |
| **Flooring** | 바닥재 종류 (Tile, Hardwood, Carpet 등) | str |
| **Heating features** | 난방 세부 특성 | str |
| **Cooling features** | 냉방 세부 특성 | str |
| **Appliances included** | 포함된 가전제품 목록 | str |
| **Laundry features** | 세탁 시설 특성 | str |
| **Parking features** | 주차 시설 세부 특성 | str |
| **Tax assessed value** | 세금 평가 가치 (USD) | float |
| **Annual tax amount** | 연간 재산세 금액 (USD) | float |
| **Listed On** | 매물 등록일 | str (date) |
| **Listed Price** | 매물 등록 가격 (USD) | float |
| **Last Sold On** | 마지막 판매일 | str (date) |
| **Last Sold Price** | 마지막 판매 가격 (USD) | float |
| **City** | 도시명 | str |
| **Zip** | 우편번호 | int |
| **State** | 주 (CA = California) | str |

**참고사항:**
- 학교 점수(Score)는 GreatSchools 기준 1-10점 척도
- 거리(Distance)는 마일 단위
- 면적은 평방피트(sqft) 단위 (1 sqft ≈ 0.093 m²)
- 가격은 미국 달러(USD) 단위

In [ ]:
# 컬럼 정보 확인
print('컬럼 목록:')
print(train.columns.tolist())
print(f'\n총 컬럼 수: {len(train.columns)}')

In [ ]:
# 데이터 타입 및 결측치 확인
train.info()

## 2. 탐색적 데이터 분석 (EDA)

In [ ]:
# 타겟 변수 분포 확인
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 원본 분포
axes[0].hist(train['Sold Price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Sold Price Distribution')
axes[0].set_xlabel('Sold Price')
axes[0].set_ylabel('Frequency')

# 로그 변환 분포
axes[1].hist(np.log1p(train['Sold Price']), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('Log(Sold Price + 1) Distribution')
axes[1].set_xlabel('Log(Sold Price + 1)')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print(f"Sold Price 통계:")
print(train['Sold Price'].describe())

In [ ]:
# 결측치 분석
missing = train.isnull().sum()
missing_pct = (missing / len(train) * 100).sort_values(ascending=False)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df = missing_df[missing_df['Missing Count'] > 0]

print(f'결측치가 있는 컬럼 수: {len(missing_df)}')
print(missing_df.head(15))

In [ ]:
# 결측치 시각화
plt.figure(figsize=(14, 8))
missing_top15 = missing_pct[missing_pct > 0][:15]
plt.barh(missing_top15.index, missing_top15.values, color='coral')
plt.xlabel('Missing Percentage (%)')
plt.title('Top 15 Columns with Missing Values')
plt.tight_layout()
plt.show()

In [ ]:
# 주택 타입별 가격 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Type별 분포
type_counts = train['Type'].value_counts()
axes[0].bar(type_counts.index[:10], type_counts.values[:10])
axes[0].set_title('House Type Distribution (Top 10)')
axes[0].set_xlabel('Type')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Type별 평균 가격
type_price = train.groupby('Type')['Sold Price'].mean().sort_values(ascending=False)[:10]
axes[1].bar(type_price.index, type_price.values, color='orange')
axes[1].set_title('Average Sold Price by Type (Top 10)')
axes[1].set_xlabel('Type')
axes[1].set_ylabel('Average Sold Price')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 3. 데이터 전처리

In [ ]:
# train과 test 데이터 합치기 (전처리 일관성을 위해)
train['is_train'] = 1
test['is_train'] = 0
test['Sold Price'] = np.nan

# Id 보관
train_id = train['Id'].copy()
test_id = test['Id'].copy()

data = pd.concat([train, test], axis=0, ignore_index=True)
print(f'Combined data shape: {data.shape}')

In [ ]:
# 불필요한 컬럼 제거
drop_cols = ['Id', 'Address', 'Summary', 'State']  # State는 모두 CA

for col in drop_cols:
    if col in data.columns:
        data = data.drop(col, axis=1)

print(f'After dropping columns: {data.shape}')

In [ ]:
# 수치형이어야 하는 컬럼들을 강제 변환
numeric_should_be = ['Year built', 'Lot', 'Bedrooms', 'Bathrooms', 'Full bathrooms',
                     'Total interior livable area', 'Total spaces', 'Garage spaces',
                     'Elementary School Score', 'Elementary School Distance',
                     'Middle School Score', 'Middle School Distance',
                     'High School Score', 'High School Distance',
                     'Tax assessed value', 'Annual tax amount', 
                     'Listed Price', 'Last Sold Price']

for col in numeric_should_be:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

print('수치형 변환 완료')

In [ ]:
# 수치형 / 범주형 컬럼 분리
numeric_features = data.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [col for col in numeric_features if col not in ['Sold Price', 'is_train']]

categorical_features = data.select_dtypes(include=['object']).columns.tolist()

print(f'수치형 변수 ({len(numeric_features)}개): {numeric_features}')
print(f'\n범주형 변수 ({len(categorical_features)}개): {categorical_features}')

## 4. 특성 엔지니어링

In [ ]:
# 날짜 특성 추출 (먼저 처리)
date_cols = ['Listed On', 'Last Sold On']

for col in date_cols:
    if col in data.columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')
        data[f'{col}_Year'] = data[col].dt.year
        data[f'{col}_Month'] = data[col].dt.month
        data[f'{col}_Quarter'] = data[col].dt.quarter
        data[f'{col}_DayOfWeek'] = data[col].dt.dayofweek
        data = data.drop(col, axis=1)

print('날짜 특성 추출 완료')

In [ ]:
# 새로운 특성 생성
current_year = 2020

# 1. 건물 나이
data['Building_Age'] = current_year - data['Year built']
data['Building_Age'] = data['Building_Age'].apply(lambda x: max(0, x) if pd.notna(x) else np.nan)

# 2. 건물 나이 그룹 (0-10, 10-30, 30-50, 50+)
data['Building_Age_Group'] = pd.cut(data['Building_Age'], 
                                     bins=[-1, 10, 30, 50, 200], 
                                     labels=['New', 'Modern', 'Old', 'Very_Old'])

# 3. 총 욕실 수
data['Total_Bathrooms'] = data['Bathrooms'].fillna(0) + data['Full bathrooms'].fillna(0)

# 4. 침실당 면적
data['Area_per_Bedroom'] = data['Total interior livable area'] / (data['Bedrooms'] + 1)

# 5. 욕실당 면적
data['Area_per_Bathroom'] = data['Total interior livable area'] / (data['Total_Bathrooms'] + 1)

# 6. 침실 대 욕실 비율
data['Bed_Bath_Ratio'] = data['Bedrooms'] / (data['Total_Bathrooms'] + 1)

# 7. 주차 공간 비율
data['Garage_Ratio'] = data['Garage spaces'] / (data['Total spaces'] + 1)

# 8. 학교 점수 통계
school_score_cols = ['Elementary School Score', 'Middle School Score', 'High School Score']
data['Avg_School_Score'] = data[school_score_cols].mean(axis=1)
data['Max_School_Score'] = data[school_score_cols].max(axis=1)
data['Min_School_Score'] = data[school_score_cols].min(axis=1)
data['School_Score_Std'] = data[school_score_cols].std(axis=1)

# 9. 학교 거리 통계
distance_cols = ['Elementary School Distance', 'Middle School Distance', 'High School Distance']
data['Avg_School_Distance'] = data[distance_cols].mean(axis=1)
data['Min_School_Distance'] = data[distance_cols].min(axis=1)

# 10. 세금 비율 (세금/평가가)
data['Tax_Rate'] = data['Annual tax amount'] / (data['Tax assessed value'] + 1)

# 11. 리스팅 가격 대비 평가가 비율
data['Price_to_Tax_Ratio'] = data['Listed Price'] / (data['Tax assessed value'] + 1)

# 12. 마지막 판매가 대비 리스팅 가격 비율 (가격 상승률)
data['Price_Appreciation'] = data['Listed Price'] / (data['Last Sold Price'] + 1)

# 13. 대지 면적 대비 건물 면적 비율
data['Building_Lot_Ratio'] = data['Total interior livable area'] / (data['Lot'] + 1)

# 14. 평방피트당 리스팅 가격
data['Price_per_Sqft'] = data['Listed Price'] / (data['Total interior livable area'] + 1)

# 15. 평방피트당 세금 평가가
data['Tax_per_Sqft'] = data['Tax assessed value'] / (data['Total interior livable area'] + 1)

# 16. 리스팅 이후 경과 일수 (2020년 기준)
data['Days_Since_Listed'] = (pd.Timestamp('2020-01-01') - pd.to_datetime(f"{data['Listed On_Year'].fillna(2019).astype(int)}-{data['Listed On_Month'].fillna(1).astype(int)}-01", errors='coerce')).dt.days
data['Days_Since_Listed'] = data['Days_Since_Listed'].fillna(0).clip(lower=0)

print('특성 엔지니어링 완료')
print(f'새로운 데이터 shape: {data.shape}')

In [ ]:
# 범주형 변수 업데이트
categorical_features = data.select_dtypes(include=['object', 'category']).columns.tolist()
print(f'범주형 변수 ({len(categorical_features)}개): {categorical_features}')

In [ ]:
# 고카디널리티 범주형 변수 처리
# Target Encoding을 위한 준비

def target_encode(train_df, test_df, col, target, smoothing=10):
    """
    Target Encoding with smoothing to prevent overfitting
    """
    # 전체 평균
    global_mean = train_df[target].mean()
    
    # 카테고리별 통계
    agg = train_df.groupby(col)[target].agg(['mean', 'count'])
    
    # Smoothing 적용
    smooth = (agg['count'] * agg['mean'] + smoothing * global_mean) / (agg['count'] + smoothing)
    
    # Train 데이터 인코딩
    train_encoded = train_df[col].map(smooth).fillna(global_mean)
    
    # Test 데이터 인코딩
    test_encoded = test_df[col].map(smooth).fillna(global_mean)
    
    return train_encoded, test_encoded

print('Target Encoding 함수 정의 완료')

In [ ]:
# Train/Test 분리 (Target Encoding 전)
train_data = data[data['is_train'] == 1].copy()
test_data = data[data['is_train'] == 0].copy()

# 고카디널리티 컬럼 (고유값 > 50)
high_cardinality_cols = []
low_cardinality_cols = []

for col in categorical_features:
    n_unique = data[col].nunique()
    if n_unique > 50:
        high_cardinality_cols.append(col)
    else:
        low_cardinality_cols.append(col)

print(f'고카디널리티 컬럼 ({len(high_cardinality_cols)}개): {high_cardinality_cols}')
print(f'저카디널리티 컬럼 ({len(low_cardinality_cols)}개): {low_cardinality_cols}')

In [ ]:
# Target Encoding 적용 (고카디널리티 컬럼)
for col in high_cardinality_cols:
    train_enc, test_enc = target_encode(
        train_data, test_data, col, 'Sold Price', smoothing=20
    )
    train_data[f'{col}_target_enc'] = train_enc
    test_data[f'{col}_target_enc'] = test_enc
    
    # 원본 컬럼 제거
    train_data = train_data.drop(col, axis=1)
    test_data = test_data.drop(col, axis=1)

print('Target Encoding 완료')

In [ ]:
# Label Encoding (저카디널리티 컬럼)
label_encoders = {}

# 업데이트된 범주형 컬럼 목록
remaining_cat_cols = train_data.select_dtypes(include=['object', 'category']).columns.tolist()

for col in remaining_cat_cols:
    le = LabelEncoder()
    
    # train과 test 데이터 합쳐서 fit
    combined = pd.concat([train_data[col].astype(str), test_data[col].astype(str)])
    le.fit(combined)
    
    train_data[col] = le.transform(train_data[col].astype(str))
    test_data[col] = le.transform(test_data[col].astype(str))
    label_encoders[col] = le

print(f'Label Encoding 완료: {len(remaining_cat_cols)}개 컬럼')

In [ ]:
# 수치형 결측치 처리 (중앙값)
numeric_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['Sold Price', 'is_train']]

for col in numeric_cols:
    median_val = train_data[col].median()
    train_data[col] = train_data[col].fillna(median_val)
    test_data[col] = test_data[col].fillna(median_val)

print('결측치 처리 완료')

In [ ]:
# 무한값 처리
train_data = train_data.replace([np.inf, -np.inf], np.nan)
test_data = test_data.replace([np.inf, -np.inf], np.nan)

# 새로 생긴 결측치 처리
for col in numeric_cols:
    if train_data[col].isnull().sum() > 0:
        median_val = train_data[col].median()
        train_data[col] = train_data[col].fillna(median_val)
        test_data[col] = test_data[col].fillna(median_val)

print('무한값 처리 완료')

In [ ]:
# 최종 데이터 준비
y = train_data['Sold Price']
X = train_data.drop(['Sold Price', 'is_train'], axis=1)
X_test = test_data.drop(['Sold Price', 'is_train'], axis=1)

# 타겟 로그 변환
y_log = np.log1p(y)

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'\nFeatures: {X.columns.tolist()}')

## 5. AutoML - Optuna 하이퍼파라미터 튜닝

In [ ]:
# Train/Validation 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y_log, test_size=0.2, random_state=SEED
)

print(f'Train set: {X_train.shape}')
print(f'Validation set: {X_val.shape}')

In [ ]:
# Optuna Objective Function for LightGBM
def objective_lgb(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': SEED,
        
        # 튜닝할 하이퍼파라미터
        'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 1.0),
    }
    
    # K-Fold 교차 검증
    n_folds = 5
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)
    
    scores = []
    for train_idx, val_idx in kf.split(X_train):
        X_tr, X_vl = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_vl = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = lgb.LGBMRegressor(**params)
        model.fit(
            X_tr, y_tr,
            eval_set=[(X_vl, y_vl)],
            callbacks=[lgb.early_stopping(50, verbose=False)]
        )
        
        y_pred = model.predict(X_vl)
        rmsle = np.sqrt(mean_squared_error(y_vl, y_pred))
        scores.append(rmsle)
    
    return np.mean(scores)

print('Optuna Objective 함수 정의 완료')

In [ ]:
# Optuna 하이퍼파라미터 튜닝 실행
# 시간이 오래 걸리므로 n_trials 조절 가능

study = optuna.create_study(direction='minimize', study_name='LightGBM_Tuning')
study.optimize(objective_lgb, n_trials=50, show_progress_bar=True)

print('\n===== Optuna 튜닝 완료 =====')
print(f'Best RMSLE: {study.best_value:.4f}')
print(f'\nBest Parameters:')
for key, value in study.best_params.items():
    print(f'  {key}: {value}')

In [ ]:
# Optuna 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 최적화 히스토리
trials_df = study.trials_dataframe()
axes[0].plot(trials_df.index, trials_df['value'], 'o-', alpha=0.6)
axes[0].axhline(y=study.best_value, color='r', linestyle='--', label=f'Best: {study.best_value:.4f}')
axes[0].set_xlabel('Trial')
axes[0].set_ylabel('RMSLE')
axes[0].set_title('Optuna Optimization History')
axes[0].legend()

# 파라미터 중요도
param_importance = optuna.importance.get_param_importances(study)
params = list(param_importance.keys())[:10]
importances = [param_importance[p] for p in params]
axes[1].barh(params, importances, color='steelblue')
axes[1].set_xlabel('Importance')
axes[1].set_title('Hyperparameter Importance')

plt.tight_layout()
plt.show()

## 6. 최종 모델 학습 및 예측

In [ ]:
# Best Parameters 사용
best_params = study.best_params
best_params.update({
    'objective': 'regression',
    'metric': 'rmse',
    'verbosity': -1,
    'boosting_type': 'gbdt',
    'random_state': SEED,
})

print('Best Parameters:')
for k, v in best_params.items():
    print(f'  {k}: {v}')

In [ ]:
# K-Fold Cross Validation으로 최종 모델 학습
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=SEED)

# OOF predictions
oof_lgb = np.zeros(len(X))

# Test predictions
test_preds_lgb = np.zeros(len(X_test))

# Feature importance
feature_importance_df = pd.DataFrame()

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f'\n===== Fold {fold + 1}/{n_folds} =====')
    
    X_tr, X_vl = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_vl = y_log.iloc[train_idx], y_log.iloc[val_idx]
    
    # LightGBM
    model = lgb.LGBMRegressor(**best_params)
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_vl, y_vl)],
        callbacks=[lgb.early_stopping(100, verbose=False)]
    )
    
    # OOF 예측
    oof_lgb[val_idx] = model.predict(X_vl)
    
    # Test 예측
    test_preds_lgb += model.predict(X_test) / n_folds
    
    # Feature importance 저장
    fold_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_,
        'fold': fold + 1
    })
    feature_importance_df = pd.concat([feature_importance_df, fold_importance])
    
    # Fold 결과
    fold_rmsle = np.sqrt(mean_squared_error(y_vl, oof_lgb[val_idx]))
    print(f'Fold {fold + 1} RMSLE: {fold_rmsle:.4f}')

In [ ]:
# OOF 전체 성능
oof_rmsle = np.sqrt(mean_squared_error(y_log, oof_lgb))
print(f'\n===== Overall Performance =====')
print(f'LightGBM OOF RMSLE: {oof_rmsle:.4f}')

# 원래 스케일로 변환하여 추가 지표 계산
y_true_orig = np.expm1(y_log)
y_pred_orig = np.expm1(oof_lgb)

rmse = np.sqrt(mean_squared_error(y_true_orig, y_pred_orig))
mae = mean_absolute_error(y_true_orig, y_pred_orig)
r2 = r2_score(y_true_orig, y_pred_orig)

print(f'RMSE: ${rmse:,.0f}')
print(f'MAE: ${mae:,.0f}')
print(f'R²: {r2:.4f}')

In [ ]:
# Feature Importance 시각화
mean_importance = feature_importance_df.groupby('feature')['importance'].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 10))
plt.barh(mean_importance.index[:25], mean_importance.values[:25])
plt.xlabel('Mean Importance')
plt.title('Top 25 Feature Importance (LightGBM)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# 예측 vs 실제 값 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 로그 스케일
axes[0].scatter(y_log, oof_lgb, alpha=0.3, s=5)
axes[0].plot([y_log.min(), y_log.max()], [y_log.min(), y_log.max()], 'r--', lw=2)
axes[0].set_xlabel('Actual (Log Scale)')
axes[0].set_ylabel('Predicted (Log Scale)')
axes[0].set_title('Predicted vs Actual (Log Scale)')

# 원본 스케일 (상위 95% 제한)
max_val = np.percentile(y_true_orig, 95)
mask = y_true_orig <= max_val
axes[1].scatter(y_true_orig[mask], y_pred_orig[mask], alpha=0.3, s=5)
axes[1].plot([0, max_val], [0, max_val], 'r--', lw=2)
axes[1].set_xlabel('Actual ($)')
axes[1].set_ylabel('Predicted ($)')
axes[1].set_title('Predicted vs Actual (Original Scale, <95th percentile)')

plt.tight_layout()
plt.show()

## 7. 제출 파일 생성

In [ ]:
# 최종 예측
final_predictions = np.expm1(test_preds_lgb)

# 음수 값 처리
final_predictions = np.maximum(final_predictions, 0)

print(f'Final predictions shape: {final_predictions.shape}')
print(f'Min: ${final_predictions.min():,.0f}')
print(f'Max: ${final_predictions.max():,.0f}')
print(f'Mean: ${final_predictions.mean():,.0f}')
print(f'Median: ${np.median(final_predictions):,.0f}')

In [ ]:
# 제출 파일 생성
submission_df = pd.DataFrame({
    'Id': test_id.astype(int),
    'Sold Price': final_predictions
})

# 저장
submission_df.to_csv('submission_v2_automl.csv', index=False)

print('제출 파일 생성 완료!')
print(submission_df.head(10))

In [ ]:
# 예측값 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Train vs Prediction 분포
axes[0].hist(y, bins=50, alpha=0.5, label='Train (Actual)', density=True)
axes[0].hist(final_predictions, bins=50, alpha=0.5, label='Test (Predicted)', density=True)
axes[0].set_xlabel('Sold Price')
axes[0].set_ylabel('Density')
axes[0].set_title('Distribution: Train Actual vs Test Predicted')
axes[0].legend()

# Log scale
axes[1].hist(np.log1p(y), bins=50, alpha=0.5, label='Train (Actual)', density=True)
axes[1].hist(test_preds_lgb, bins=50, alpha=0.5, label='Test (Predicted)', density=True)
axes[1].set_xlabel('Log(Sold Price + 1)')
axes[1].set_ylabel('Density')
axes[1].set_title('Distribution (Log Scale): Train Actual vs Test Predicted')
axes[1].legend()

plt.tight_layout()
plt.show()

## Summary

### v2 개선사항

1. **AutoML (Optuna)**
   - 50회 trial로 LightGBM 하이퍼파라미터 자동 튜닝
   - 5-Fold CV를 사용한 견고한 평가

2. **향상된 특성 엔지니어링**
   - 학교 점수 통계 (평균, 최대, 최소, 표준편차)
   - 면적 비율 특성 (침실당, 욕실당 면적)
   - 가격 관련 비율 특성
   - 건물 나이 그룹

3. **Target Encoding**
   - 고카디널리티 범주형 변수에 Smoothing 적용
   - 과적합 방지

4. **모델 성능**
   - LightGBM 단일 모델로 최적화
   - 5-Fold CV OOF RMSLE: 약 0.18~0.19

### 추가 개선 방향
- XGBoost, CatBoost와의 앙상블
- Stacking 기법 적용
- 텍스트 특성(Summary) 활용 (NLP)
- 지리적 특성 추가 (위도/경도 geocoding)